In [1]:
# Standard library imports
import os
from datetime import datetime
import ast

# Third-party library imports
import pandas as pd
from dotenv import load_dotenv
import numpy as np
import re
import asyncio

load_dotenv()

True

In [2]:
from typing import Any, List, Tuple, Optional, Dict, Union
# Local application/library specific imports
# from langchain.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [3]:
# Load transactions data to categorise
tx_dir = 'C:/Users/Barry/OneDrive/Documents/My stuff/Personal finance/expense_manager/data/tx_data/'
tx_list = pd.read_csv(os.path.join(tx_dir, 'output/tx_master_data.csv'))

##### LLM categorise transactions

In [4]:
# Load prompt
EXPENSE_CAT_TEMPLATE = """
    <context>
    You are an advanced data analysis model. Your task is to create a list of [description, category] lists, where in each list item:
    - the description is exactly the same you receive as input
    - the category of the transaction (choose one from the list below based on the description)
    </context>
    
    <categories>
    The following list contains the categories and associated keywords you often see in transaction descriptions:
    -ATM: atm, cash, withdraw
    -Auto: auto body
    -Bars: bar, pubs, irish, brewery
    -Beauty: body
    -Cashback: cashback, reward, bonus, cash back
    -Clothing: clothing, shoes, accessories
    -Coffee Shops: coffee, cafe, tea, Starbucks, Dunkin
    -Credit Card Payment: card payment, autopay
    -Education: kindle, tuition
    -Entertainment: event, show, movies, cinema, theater
    -Fees: Fee
    -Food: snack, Donalds, Burger King, KFC, Subway, Pizza, Domino, Taco Bell, Wendy, Chick-fil-A, Popeyes, Arby's, Chipotle, or someone's name (e.g. Harris)
    -Fuel: fuel, gas, petrol
    -Gifts: donation, gift
    -Groceries: groceries, supermarket, food, familia
    -Gym: gym, fitness, yoga, pilates, crossfit
    -Home: Ikea
    -Housing: rent, mortgage
    -Income: refund, deposit, paycheck
    -Insurance: insurance
    -Medical: medical, doctor, dentist, hospital, clinic
    -Pets: vet, veterinary, pet, dog, cat
    -Pharmacy: pharmacy, drugstore, cvs, walgreens, rite aid, duane
    -Restaurants: restaurant, lunch, dinner
    -Services: service, laundry, dry cleaning
    -Shopping: shopping, amazon, walmart, target, safeway
    -Streaming: Netflix, Spotify, Hulu, HBO
    -Taxes: tax, irs
    -Technology: technology, software, hardware, electronics
    -Transportation: bus, train, subway, metro, airline, uber, lyft, taxi
    -Travel: travel, holiday, trip, airbnb, kiwi, hotel, hostel, resort, kiwi, kayak, expedia, booking.com
    -Transfer: payment from
    -Utilities: electricity, water, gas, ting, verizon, comcast, sprint, t-mobile, at&t, mint
    -Other: use this when very uncertain about the category
    </categories>

    <formatting_instructions>
    Your output should be a valid list of lists (e.g. [[description1, category1], [description2, category2], ...]] parse-able by the command ast.literal_eval(output)
    Don't include any kind of commentary, return carriages, spaces or other characters in the output
    Fill all categories; if you don't know which one to choose, choose 'Other'
    </formatting_instructions>
    
    
    <financial_transactions>
    {input_data}
    </financial_transactions>"""

In [5]:
# Initialize language model and prompt
llm = ChatOpenAI(temperature=0, client=Any)
prompt = PromptTemplate.from_template(template=EXPENSE_CAT_TEMPLATE)    
chain = LLMChain(llm=llm, prompt=prompt)

c:\Users\Barry\dev\venvs\expense_llm\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\Barry\dev\venvs\expense_llm\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [24]:
# Iterate over the DataFrame in batches of TX_PER_LLM_RUN transactions
TX_PER_LLM_RUN = 10

def validate_result(raw_result):
    result = {'valid': True, 'output': []}

    try:
        # Create a pattern to match a list Description-Category pairs (List[Tuple[str, str]])
        pattern = r"\['([^']+)', '([^']+)'\]"
        
        # Use it to extract all the correctly formatted pairs from the raw result
        matches = re.findall(pattern, raw_result.replace("\\'", "'"))

        # Loop over the matches, and try to parse them to ensure the content is valid
        valid_outputs = []
        for match in matches:
            try:
                parsed_pair = ast.literal_eval(str(list(match)))
                valid_outputs.append(parsed_pair)
            except Exception as e:
                # logger.log(logging.ERROR, f"Parsing Error: {e}\nMatch: {match}\n")
                result['valid'] = False

        result['output'] = valid_outputs

    except Exception as e:
        result['valid'] = False

async def llm_categorise_chunk(chunk, chain):
    input_string = "\n".join(chunk['description']).strip()
    
    # categorise
    raw_result = await chain.arun(input_data=input_string)
    
    # validate result
    # result = validate_result(raw_result)

    return raw_result


In [26]:
# def llm_categorise(tx_list, chain, TX_PER_LLM_RUN):
chunks = np.array_split(tx_list, tx_list.shape[0] // TX_PER_LLM_RUN + 1)

len(chunks)

c:\Users\Barry\dev\venvs\expense_llm\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


81

In [29]:
import math
math.floor(len(chunks)/5)

16

In [44]:
# Define number of chunks to run per iteration (bound of org limit on tokens)
no_chunks = 5
iterations = math.floor(len(chunks)/no_chunks)

# adjust for remainder of chunks
if len(chunks)/no_chunks != iterations:
    iterations += 1

final_results_df = pd.DataFrame()

for i in range(iterations):
    start = i*5
    stop = np.where((i+1)*5 > len(chunks), len(chunks), (i+1)*5)
    print(start, stop)
    
    # categorise chunk with prompt in ChatGPT
    results = [llm_categorise_chunk(chunk, chain) for chunk in chunks[start:stop]] 
    all_results = await asyncio.gather(*results)
    
    # organise results in dataframe
    results_df = pd.concat([pd.DataFrame(ast.literal_eval(df), columns=['description', 'category']) for df in all_results], ignore_index=True)
    final_results_df = pd.concat([final_results_df, results_df])

0 5
5 10
10 15
15 20
20 25
25 30
30 35
35 40
40 45
45 50
50 55
55 60
60 65
65 70


SyntaxError: unterminated string literal (detected at line 1) (<unknown>, line 1)

In [46]:
i

13

In [49]:
start = i*5
stop = np.where((i+1)*5 > len(chunks), len(chunks), (i+1)*5)

result_dict = {}

for q, chunk in enumerate(chunks[start:stop]):
    result_dict[q] = llm_categorise_chunk(chunk, chain)
    all_results = await asyncio.gather(*result_dict)

In [60]:
all_results

["[['HMART GROUP PTY LTD EASTWOOD', 'Groceries'], ['SAVOY CAKE EASTWOOD Eastwood', 'Food'], ['THAI KEE IGA SUPERMA HAYMARKET', 'Groceries'], ['WOOLWORTHS/ROWE ST EASTWOOD', 'Groceries'], ['Costis Fish and Chips Sydney', 'Food'], ['DAVELLES BAKERY EPPING', 'Food'], ['EDITION ROASTERS MC Sydney', 'Coffee Shops'], ['YY CAFE PTY LTD EASTWOOD', 'Coffee Shops'], ['HEALTH SQ EPPING', 'Medical'], ['Machi Machi Eastwood Eastwood', 'Coffee Shops']]",
 '[[\'THE MAX JOY CO EASTWOOD\', \'Other\'], [\'HEALTH SQ EPPING\', \'Medical\'], [\'TAN VIET DARLG SQ PL HAYMARKET\', \'Other\'], [\'RETAIL JUICE BARS PTY ULTIMO\', \'Bars\'], [\'THE Q ON HARRIS Ultimo\', \'Other\'], [\'THE MISSING PIECE FRENCHS FORES\', \'Other\'], [\'COLES 0839 BROADWAY\', \'Groceries\'], ["CAT & CLAUDIA\'S PCY Broadway", \'Pets\'], [\'SQ *LIM\'S HOT BREAD Eastwood\', \'Other\'], [\'YY CAFE PTY LTD EASTWOOD\', \'Coffee Shops\']]',
 "[['PAYPAL *NSWBJJFEDER 4029357733', 'Other'], ['Boom Cafe Eastwood', 'Coffee Shops'], ['PAYPAL *BO

In [65]:
tx_list

,date,description,amount,provider
0,25/05/2024,NORTH SANDWICH&BURGER,-10.69,AMEX
1,25/05/2024,NORTH SANDWICH&BURGER 6 CHIPPENDALE,53.45,AMEX
2,25/05/2024,POCKET ROCKETZ Ultimo,36.48,AMEX
3,25/05/2024,PROVENANCE BY ANDREW BA Ultimo,36.48,AMEX
4,25/05/2024,SHERIDAN OUTLET KEW,26.00,AMEX
...,...,...,...,...
803,02 Oct 23,PULP+GRIND SYD T2 Mascot,-14.05,NAB
804,02 Oct 23,EDITION ROASTERS MC Sydney,-5.00,NAB
805,29 Sep 23,Machi Machi Eastwood Eastwood,-8.12,NAB
806,28 Sep 23,NOMADIC ID LONDON,-783.08,NAB


In [77]:
final_results_df.shape

(648, 2)

In [79]:
len(chunks[0])

10

In [82]:
tx_list.loc[tx_list.description == "CAT & CLAUDIA'S PCY Broadway"]

,date,description,amount,provider
667,20 Nov 23,CAT & CLAUDIA'S PCY Broadway,-12.99,NAB


In [83]:
chunks[66]

,date,description,amount,provider
660,22 Nov 23,THE MAX JOY CO EASTWOOD,-6.09,NAB
661,21 Nov 23,HEALTH SQ EPPING,-97.00,NAB
662,21 Nov 23,TAN VIET DARLG SQ PL HAYMARKET,-39.33,NAB
663,21 Nov 23,RETAIL JUICE BARS PTY ULTIMO,-8.90,NAB
664,21 Nov 23,THE Q ON HARRIS Ultimo,-5.00,NAB
665,20 Nov 23,THE MISSING PIECE FRENCHS FORES,-76.30,NAB
666,20 Nov 23,COLES 0839 BROADWAY,-48.00,NAB
667,20 Nov 23,CAT & CLAUDIA'S PCY Broadway,-12.99,NAB
668,20 Nov 23,SQ *LIM'S HOT BREAD Eastwood,-8.10,NAB
669,20 Nov 23,YY CAFE PTY LTD EASTWOOD,-4.50,NAB


In [86]:
chunks[64]

,date,description,amount,provider
640,28 Nov 23,J & R 99 PTY LTD HAYMARKET,-8.32,NAB
641,28 Nov 23,YY CAFE PTY LTD EASTWOOD,-5.80,NAB
642,28 Nov 23,TRANSPORTFORNSW TAP SYDNEY,-2.24,NAB
643,27 Nov 23,PAYPAL *THEGOODGUYS 4029357733,-145.00,NAB
644,27 Nov 23,PAYPAL *LOBOCKI 4029357733,-86.98,NAB
645,27 Nov 23,RUNGNAPA KORNPAN WEST RYDE,-75.86,NAB
646,27 Nov 23,SAVOY CAKE EASTWOOD Eastwood,-37.74,NAB
647,27 Nov 23,SQ *KAPPUCCINO Ultimo,-32.00,NAB
648,27 Nov 23,PAYPAL *BIG W 4029357733,-24.00,NAB
649,27 Nov 23,OFFICEWORKS 0230 WEST RYDE,-22.69,NAB


In [84]:
final_results_df.iloc[-5:] # ran up to chunk 64

,description,category
45,RUNGNAPA KORNPAN WEST RYDE,Food
46,SAVOY CAKE EASTWOOD Eastwood,Food
47,SQ *KAPPUCCINO Ultimo,Coffee Shops
48,PAYPAL *BIG W 4029357733,Shopping
49,OFFICEWORKS 0230 WEST RYDE,Shopping


In [88]:
i*5

65

In [94]:
# save results so far
final_results_df_12iter = final_results_df.copy()

In [96]:
# problem: 'SQ *LIM'S HOT BREAD Eastwood'
# why is python not using double quotes?
tx_list.loc[tx_list.description == "SQ *LIM'S HOT BREAD Eastwood"]

,date,description,amount,provider
668,20 Nov 23,SQ *LIM'S HOT BREAD Eastwood,-8.1,NAB
693,08 Nov 23,SQ *LIM'S HOT BREAD Eastwood,-10.2,NAB
753,19 Oct 23,SQ *LIM'S HOT BREAD Eastwood,-12.2,NAB


In [95]:
# run remaining chunks from 65 to 81
for i in range(14,17): # remove 13
    start = i*5
    stop = np.where((i+1)*5 > len(chunks), len(chunks), (i+1)*5)
    print(start, stop)
    
    # categorise chunk with prompt in ChatGPT
    results = [llm_categorise_chunk(chunk, chain) for chunk in chunks[start:stop]] 
    all_results = await asyncio.gather(*results)
    
    # organise results in dataframe
    results_df = pd.concat([pd.DataFrame(ast.literal_eval(df), columns=['description', 'category']) for df in all_results], ignore_index=True)
    final_results_df = pd.concat([final_results_df, results_df])

70 75
75 80


SyntaxError: unterminated string literal (detected at line 1) (<unknown>, line 1)

In [58]:
final_results_df.category.value_counts()

category
Coffee Shops      121
Other              87
Shopping           76
Restaurants        71
Groceries          69
Food               47
Medical            21
Transportation     19
Beauty             13
Technology         13
Streaming          12
Pharmacy           12
Clothing           11
Transfer            8
Fees                8
Home                7
Education           7
Utilities           6
Fuel                5
Insurance           5
Services            5
Entertainment       5
Gym                 4
Pets                3
Auto                3
Travel              3
Cashback            2
Restaurant          2
Alcohol             1
Bars                1
Housing             1
Name: count, dtype: int64

##### Validate results

In [68]:
# check if different categories are classified for the same description
dups = final_results_df.groupby('description').category.nunique()[final_results_df.groupby('description').category.nunique() > 1]
dups = dups.reset_index()
dups.columns = ['description', 'no_categories']

In [76]:
pd.merge(final_results_df, dups, left_on='description', right_on='description')

,description,category,no_categories
0,THE Q ON HARRIS 58996 ULTIMO,Other,2
1,THE Q ON HARRIS 58996 ULTIMO,Other,3
2,THE Q ON HARRIS 58996 ULTIMO,Housing,3
3,PANTRY STORY STANMORE,Groceries,2
4,THE Q ON HARRIS 58996 ULTIMO,Other,3
...,...,...,...
82,NOMADIC ID LONDON,Travel,2
83,PAYPAL *LOBOCKI 4029357733,Other,2
84,NOMADIC ID LONDON,Other,2
85,PAYPAL *LOBOCKI 4029357733,Shopping,2


##### Single run

In [25]:
input_string = "\n".join(chunks[0]['description']).strip()

# categorise
raw_result = await chain.arun(input_data=input_string)

# validate result
result = validate_result(raw_result)

In [29]:
raw_result

"[['NORTH SANDWICH&BURGER', 'Food'], ['NORTH SANDWICH&BURGER 6 CHIPPENDALE', 'Food'], ['POCKET ROCKETZ', 'Other'], ['PROVENANCE BY ANDREW BA', 'Other'], ['SHERIDAN OUTLET', 'Shopping'], ['SMOKEY SUE\\'S', 'Food'], ['THE Q ON HARRIS', 'Other'], ['THE Q ON HARRIS 58996', 'Other'], ['GRAM CAFE THE GALERIES', 'Coffee Shops'], ['GRAM CAFE THE GALERIES  Sydney', 'Other']]"

In [28]:
pattern = r"\['([^']+)', '([^']+)'\]"
result = {'valid': True, 'output': []}        
# Use it to extract all the correctly formatted pairs from the raw result
matches = re.findall(pattern, raw_result.replace("\\'", "'"))

valid_outputs = []
for match in matches:
    try:
        parsed_pair = ast.literal_eval(str(list(match)))
        valid_outputs.append(parsed_pair)
    except Exception as e:
        # logger.log(logging.ERROR, f"Parsing Error: {e}\nMatch: {match}\n")
        result['valid'] = False

result['output'] = valid_outputs

result

{'valid': True,
 'output': [['NORTH SANDWICH&BURGER', 'Food'],
  ['NORTH SANDWICH&BURGER 6 CHIPPENDALE', 'Food'],
  ['POCKET ROCKETZ', 'Other'],
  ['PROVENANCE BY ANDREW BA', 'Other'],
  ['SHERIDAN OUTLET', 'Shopping'],
  ['THE Q ON HARRIS', 'Other'],
  ['THE Q ON HARRIS 58996', 'Other'],
  ['GRAM CAFE THE GALERIES', 'Coffee Shops'],
  ['GRAM CAFE THE GALERIES  Sydney', 'Other']]}

In [46]:
result = {'valid': True, 'output': []}

try:
    # Create a pattern to match a list Description-Category pairs (List[Tuple[str, str]])
    pattern = r"\['([^']+)', '([^']+)'\]"

    # Use it to extract all the correctly formatted pairs from the raw result
    matches = re.findall(pattern, raw_result.replace("\\'", "'"))

    # Loop over the matches, and try to parse them to ensure the content is valid
    valid_outputs = []
    for match in matches:
        try:
            parsed_pair = ast.literal_eval(str(list(match)))
            valid_outputs.append(parsed_pair)
        except Exception as e:
            # logger.log(logging.ERROR, f"Parsing Error: {e}\nMatch: {match}\n")
            result['valid'] = False

    result['output'] = valid_outputs
except Exception as e:
        result['valid'] = False    

In [47]:
valid_outputs

[['NORTH SANDWICH&BURGER', 'Food'],
 ['NORTH SANDWICH&BURGER 6 CHIPPENDALE', 'Food'],
 ['POCKET ROCKETZ', 'Other'],
 ['PROVENANCE BY ANDREW BA', 'Other'],
 ['SHERIDAN OUTLET', 'Shopping'],
 ['THE Q ON HARRIS', 'Other'],
 ['THE Q ON HARRIS 58996', 'Other'],
 ['GRAM CAFE THE GALERIES', 'Coffee Shops'],
 ['GRAM CAFE THE GALERIES  Sydney', 'Other']]

In [51]:
result['output'] = valid_outputs

In [52]:
result

{'valid': False,
 'output': [['NORTH SANDWICH&BURGER', 'Food'],
  ['NORTH SANDWICH&BURGER 6 CHIPPENDALE', 'Food'],
  ['POCKET ROCKETZ', 'Other'],
  ['PROVENANCE BY ANDREW BA', 'Other'],
  ['SHERIDAN OUTLET', 'Shopping'],
  ['THE Q ON HARRIS', 'Other'],
  ['THE Q ON HARRIS 58996', 'Other'],
  ['GRAM CAFE THE GALERIES', 'Coffee Shops'],
  ['GRAM CAFE THE GALERIES  Sydney', 'Other']]}